In [1]:
import pandas as pd
import numpy as np
import os
import time
import collections


from sklearn.model_selection import train_test_split 
import xgboost as xgb

In [2]:
#Dask
import dask.dataframe as dd
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard

client

Client Scheduler: tcp://127.0.0.1:52126 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 34.31 GB


                 Test      Evaluation  Delta  
ICL_100F        .3645%     .3267%      
ICL_ICC         .3645%     .3490%
ICL_ICC_100F    .3645%     .3142% <-
100F            .3645%     .3283%
ICC_100F        .3645%     .3138% <-

ICL = Independent Claim Length
ICC = Independent Claim Count
100f = 100 languange features from doc2vec model



In [ ]:
###Results
{'ICL_100F':(.3645 - .3267)*100,      
'ICL_ICC':(.3645 - .3490)*100,
'ICL_ICC_100F':(.3645 - .3142)*100,
'100F':(.3645 - .3283)*100,
'ICC_100F':(.3645 - .3138)*100}

In [3]:
#os.chdir('Patent Claims')

os.chdir('D:\Datasets')

In [4]:
## this way of loading in datasets one is outdated but works fine, this was done before I bought more RAM
## I couldn't fit everything in memory look to the primary claim experiments for better automated code

# Model ICL_100F
#df= dd.read_csv('claims_vectors_litigation.csv',usecols=lambda x : x not in ['claim_txt','ICC'])

# Model ICC_100F
#df= dd.read_csv('claims_vectors_litigation.csv',usecols=lambda x : x not in ['claim_txt','ICL'])


# Model ICL_ICC
#df= dd.read_csv('claims_vectors_litigation.csv',usecols=['claim_no','ICL','ICC','litigation'])

#Model ICL_ICF_100F
df= dd.read_csv('claims_vectors_litigation.csv',usecols=lambda x : x not in ['claim_txt'])

# Model 100F
#df= dd.read_csv('claims_vectors_litigation.csv',usecols=lambda x : x not in ['ICL','ICC','claim_txt'])

In [13]:
df['litigation'].value_counts().compute()

0    8185679
1      30352
Name: litigation, dtype: int64

In [5]:
%%time
### This will take forever
#df = pd.read_csv('claims_vectors_litigation.csv',usecols=lambda x : x not in ['claim_txt'])

Wall time: 0 ns


In [5]:
%%time
df = df.compute()

Wall time: 2min 39s


In [6]:
df.head()

,claim_no,ICL,ICC,litigation,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,8697278-17,106,2,0,-0.021915,0.124170,-0.467477,1.725972,1.353391,-0.197958,...,-1.201108,-0.272672,0.649778,-1.023098,0.146432,0.122571,0.215042,-1.076661,-0.508896,-1.690101
1,8697278-1,97,2,0,-0.271349,0.260269,-0.315466,1.746054,0.874254,-0.225060,...,-1.190130,-0.417508,0.587534,-1.157797,0.314950,0.189521,0.474607,-1.119020,-0.376173,-1.699343
2,7385756-81,108,33,0,1.429105,0.936555,-0.296551,-1.073354,-0.421059,-0.114407,...,0.885844,0.934247,0.678008,-0.752642,-0.121980,-0.623260,-2.384536,0.471751,0.874406,1.251178
3,7385756-94,116,33,0,1.027478,0.484034,-0.535233,-1.124252,-0.382812,-0.101879,...,0.772912,0.951951,0.901475,-0.721144,-0.304449,-0.278908,-2.655865,0.323664,0.943390,0.687369
4,7385756-79,103,33,0,1.396978,0.419591,-0.072397,-0.642861,-0.482944,0.496666,...,0.074441,1.187986,0.909484,0.278953,-0.483475,-0.028059,-2.268470,0.986993,0.039141,0.374451


In [13]:
df.loc[:,['ICL','litigation']].groupby(['litigation']).agg(['count','min','max','mean','std'])

ICL                                  
              count min    max        mean        std
litigation                                           
0           8185679   1  18433  104.827044  73.717400
1             30352   1   1509   94.007380  62.130354

In [18]:
df.loc[df['ICL']<3,]

,claim_no,ICL,ICC,litigation
8917,6229491-18,1,14,0
14641,7070712-14,2,13,0
14642,7070712-4,2,13,0
14643,7070712-3,1,13,0
14645,7070712-10,2,13,0
14647,7070712-12,1,13,0
14648,7070712-9,1,13,0
14651,7070712-8,1,13,0
14652,7070712-11,1,13,0
14653,7070712-13,1,13,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8216031 entries, 0 to 17030
Columns: 103 entries, claim_no to 99
dtypes: float64(100), int64(2), object(1)
memory usage: 6.4+ GB


In [8]:
df.index = df['claim_no']

In [14]:
df.head()

,claim_no,ICL,ICC,litigation
0,8697278-17,106,2,0
1,8697278-1,97,2,0
2,7385756-81,108,33,0
3,7385756-94,116,33,0
4,7385756-79,103,33,0


In [10]:
y = df['litigation']
X = df.drop(['litigation','claim_no'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=42)

#free up memory
del df,y,X

In [11]:
y_test.value_counts(normalize=True)

0    0.996355
1    0.003645
Name: litigation, dtype: float64

In [12]:
y_train.value_counts(normalize=True)

0    0.996285
1    0.003715
Name: litigation, dtype: float64

In [13]:
# Convert input data from numpy to XGBoost format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

#again to conserve memory
del X_train, X_test, y_train, y_test

In [14]:
# Specify sufficient boosting iterations to reach a minimum
num_round = 5000

param = {'objective'   : 'binary:logistic', # Specify binary classification
         'eta'         : .5,
         'max_depth'   : 8,
         'predictor'   : 'gpu_predictor',
         #'verbosity'   : 3,
         'tree_method' : 'gpu_hist', # Use GPU accelerated algorithm
         'random_state': 42
        }

gpu_res = {} # Store accuracy result
tmp = time.time()
# Train model
tst_model = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res)
print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))

[0]	test-error:0.003481
[1]	test-error:0.003471
[2]	test-error:0.00348
[3]	test-error:0.003476
[4]	test-error:0.003475
[5]	test-error:0.003472
[6]	test-error:0.00347
[7]	test-error:0.003471
[8]	test-error:0.00347
[9]	test-error:0.003466
[10]	test-error:0.003468
[11]	test-error:0.003466
[12]	test-error:0.003468
[13]	test-error:0.003467
[14]	test-error:0.003467
[15]	test-error:0.003465
[16]	test-error:0.003465
[17]	test-error:0.003462
[18]	test-error:0.00346
[19]	test-error:0.003459
[20]	test-error:0.003453
[21]	test-error:0.003452
[22]	test-error:0.003451
[23]	test-error:0.003453
[24]	test-error:0.003452
[25]	test-error:0.003449
[26]	test-error:0.003449
[27]	test-error:0.003443
[28]	test-error:0.003446
[29]	test-error:0.003445
[30]	test-error:0.003444
[31]	test-error:0.003447
[32]	test-error:0.003443
[33]	test-error:0.003445
[34]	test-error:0.003439
[35]	test-error:0.003432
[36]	test-error:0.00342
[37]	test-error:0.003421
[38]	test-error:0.003415
[39]	test-error:0.003412
[40]	test-error

[321]	test-error:0.003261
[322]	test-error:0.00326
[323]	test-error:0.003261
[324]	test-error:0.003257
[325]	test-error:0.003259
[326]	test-error:0.003259
[327]	test-error:0.003258
[328]	test-error:0.003258
[329]	test-error:0.003259
[330]	test-error:0.00326
[331]	test-error:0.003258
[332]	test-error:0.003257
[333]	test-error:0.003259
[334]	test-error:0.003255
[335]	test-error:0.003255
[336]	test-error:0.003253
[337]	test-error:0.003251
[338]	test-error:0.003252
[339]	test-error:0.003253
[340]	test-error:0.003252
[341]	test-error:0.003251
[342]	test-error:0.003252
[343]	test-error:0.003248
[344]	test-error:0.003247
[345]	test-error:0.003247
[346]	test-error:0.003249
[347]	test-error:0.003249
[348]	test-error:0.00325
[349]	test-error:0.003245
[350]	test-error:0.003245
[351]	test-error:0.003244
[352]	test-error:0.003245
[353]	test-error:0.003244
[354]	test-error:0.003243
[355]	test-error:0.003243
[356]	test-error:0.003244
[357]	test-error:0.003244
[358]	test-error:0.003243
[359]	test-erro

[638]	test-error:0.003198
[639]	test-error:0.003195
[640]	test-error:0.003197
[641]	test-error:0.003195
[642]	test-error:0.003197
[643]	test-error:0.003197
[644]	test-error:0.003198
[645]	test-error:0.003199
[646]	test-error:0.003199
[647]	test-error:0.003198
[648]	test-error:0.003198
[649]	test-error:0.003197
[650]	test-error:0.003198
[651]	test-error:0.003199
[652]	test-error:0.003198
[653]	test-error:0.003199
[654]	test-error:0.003199
[655]	test-error:0.003198
[656]	test-error:0.003197
[657]	test-error:0.003198
[658]	test-error:0.003197
[659]	test-error:0.003199
[660]	test-error:0.003198
[661]	test-error:0.003198
[662]	test-error:0.003198
[663]	test-error:0.003197
[664]	test-error:0.003197
[665]	test-error:0.003197
[666]	test-error:0.003198
[667]	test-error:0.003199
[668]	test-error:0.003197
[669]	test-error:0.003199
[670]	test-error:0.003198
[671]	test-error:0.003198
[672]	test-error:0.003198
[673]	test-error:0.003198
[674]	test-error:0.003197
[675]	test-error:0.003198
[676]	test-e

[955]	test-error:0.003172
[956]	test-error:0.003173
[957]	test-error:0.003172
[958]	test-error:0.003171
[959]	test-error:0.003171
[960]	test-error:0.003173
[961]	test-error:0.003171
[962]	test-error:0.003171
[963]	test-error:0.003171
[964]	test-error:0.003171
[965]	test-error:0.003171
[966]	test-error:0.00317
[967]	test-error:0.003169
[968]	test-error:0.00317
[969]	test-error:0.003169
[970]	test-error:0.00317
[971]	test-error:0.003168
[972]	test-error:0.003168
[973]	test-error:0.003169
[974]	test-error:0.003168
[975]	test-error:0.003168
[976]	test-error:0.003169
[977]	test-error:0.003169
[978]	test-error:0.003168
[979]	test-error:0.003168
[980]	test-error:0.003168
[981]	test-error:0.003168
[982]	test-error:0.003167
[983]	test-error:0.003165
[984]	test-error:0.003165
[985]	test-error:0.003167
[986]	test-error:0.003167
[987]	test-error:0.003166
[988]	test-error:0.003167
[989]	test-error:0.003166
[990]	test-error:0.003165
[991]	test-error:0.003165
[992]	test-error:0.003164
[993]	test-erro

[1262]	test-error:0.003155
[1263]	test-error:0.003155
[1264]	test-error:0.003156
[1265]	test-error:0.003156
[1266]	test-error:0.003157
[1267]	test-error:0.003157
[1268]	test-error:0.003157
[1269]	test-error:0.003158
[1270]	test-error:0.003156
[1271]	test-error:0.003158
[1272]	test-error:0.003158
[1273]	test-error:0.003158
[1274]	test-error:0.003157
[1275]	test-error:0.003158
[1276]	test-error:0.003157
[1277]	test-error:0.003155
[1278]	test-error:0.003156
[1279]	test-error:0.003157
[1280]	test-error:0.003156
[1281]	test-error:0.003154
[1282]	test-error:0.003156
[1283]	test-error:0.003158
[1284]	test-error:0.003156
[1285]	test-error:0.003155
[1286]	test-error:0.003155
[1287]	test-error:0.003157
[1288]	test-error:0.003157
[1289]	test-error:0.003156
[1290]	test-error:0.003156
[1291]	test-error:0.003156
[1292]	test-error:0.003156
[1293]	test-error:0.003153
[1294]	test-error:0.003156
[1295]	test-error:0.003155
[1296]	test-error:0.003156
[1297]	test-error:0.003156
[1298]	test-error:0.003156
[

...

[3698]	test-error:0.003143
[3699]	test-error:0.003142
[3700]	test-error:0.003142
[3701]	test-error:0.003141
[3702]	test-error:0.003141
[3703]	test-error:0.003141
[3704]	test-error:0.003141
[3705]	test-error:0.003142
[3706]	test-error:0.003142
[3707]	test-error:0.003142
[3708]	test-error:0.003142
[3709]	test-error:0.003142
[3710]	test-error:0.003142
[3711]	test-error:0.003142
[3712]	test-error:0.003143
[3713]	test-error:0.003143
[3714]	test-error:0.003143
[3715]	test-error:0.003143
[3716]	test-error:0.003143
[3717]	test-error:0.003143
[3718]	test-error:0.003143
[3719]	test-error:0.003142
[3720]	test-error:0.003143
[3721]	test-error:0.003142
[3722]	test-error:0.003143
[3723]	test-error:0.003143
[3724]	test-error:0.003142
[3725]	test-error:0.003142
[3726]	test-error:0.003142
[3727]	test-error:0.003142
[3728]	test-error:0.003142
[3729]	test-error:0.003142
[3730]	test-error:0.003142
[3731]	test-error:0.003142
[3732]	test-error:0.003142
[3733]	test-error:0.003142
[3734]	test-error:0.003142
[

[4002]	test-error:0.003139
[4003]	test-error:0.003139
[4004]	test-error:0.003139
[4005]	test-error:0.003139
[4006]	test-error:0.003139
[4007]	test-error:0.00314
[4008]	test-error:0.003141
[4009]	test-error:0.00314
[4010]	test-error:0.00314
[4011]	test-error:0.00314
[4012]	test-error:0.003141
[4013]	test-error:0.00314
[4014]	test-error:0.003141
[4015]	test-error:0.003141
[4016]	test-error:0.003141
[4017]	test-error:0.00314
[4018]	test-error:0.00314
[4019]	test-error:0.00314
[4020]	test-error:0.00314
[4021]	test-error:0.00314
[4022]	test-error:0.00314
[4023]	test-error:0.003141
[4024]	test-error:0.003141
[4025]	test-error:0.003141
[4026]	test-error:0.003141
[4027]	test-error:0.00314
[4028]	test-error:0.00314
[4029]	test-error:0.003141
[4030]	test-error:0.00314
[4031]	test-error:0.00314
[4032]	test-error:0.00314
[4033]	test-error:0.00314
[4034]	test-error:0.00314
[4035]	test-error:0.00314
[4036]	test-error:0.00314
[4037]	test-error:0.00314
[4038]	test-error:0.00314
[4039]	test-error:0.003

[4309]	test-error:0.00314
[4310]	test-error:0.00314
[4311]	test-error:0.00314
[4312]	test-error:0.003139
[4313]	test-error:0.00314
[4314]	test-error:0.003139
[4315]	test-error:0.003141
[4316]	test-error:0.003141
[4317]	test-error:0.00314
[4318]	test-error:0.00314
[4319]	test-error:0.00314
[4320]	test-error:0.00314
[4321]	test-error:0.00314
[4322]	test-error:0.00314
[4323]	test-error:0.00314
[4324]	test-error:0.003141
[4325]	test-error:0.00314
[4326]	test-error:0.003141
[4327]	test-error:0.003141
[4328]	test-error:0.003141
[4329]	test-error:0.003141
[4330]	test-error:0.003141
[4331]	test-error:0.003141
[4332]	test-error:0.003141
[4333]	test-error:0.003141
[4334]	test-error:0.003141
[4335]	test-error:0.003141
[4336]	test-error:0.00314
[4337]	test-error:0.00314
[4338]	test-error:0.00314
[4339]	test-error:0.00314
[4340]	test-error:0.00314
[4341]	test-error:0.00314
[4342]	test-error:0.00314
[4343]	test-error:0.00314
[4344]	test-error:0.003139
[4345]	test-error:0.003139
[4346]	test-error:0.0

[4616]	test-error:0.00314
[4617]	test-error:0.00314
[4618]	test-error:0.00314
[4619]	test-error:0.00314
[4620]	test-error:0.00314
[4621]	test-error:0.00314
[4622]	test-error:0.00314
[4623]	test-error:0.003139
[4624]	test-error:0.00314
[4625]	test-error:0.00314
[4626]	test-error:0.003139
[4627]	test-error:0.00314
[4628]	test-error:0.00314
[4629]	test-error:0.003139
[4630]	test-error:0.003139
[4631]	test-error:0.003139
[4632]	test-error:0.00314
[4633]	test-error:0.003139
[4634]	test-error:0.003139
[4635]	test-error:0.003139
[4636]	test-error:0.00314
[4637]	test-error:0.003139
[4638]	test-error:0.00314
[4639]	test-error:0.00314
[4640]	test-error:0.00314
[4641]	test-error:0.003139
[4642]	test-error:0.00314
[4643]	test-error:0.00314
[4644]	test-error:0.00314
[4645]	test-error:0.003139
[4646]	test-error:0.00314
[4647]	test-error:0.003139
[4648]	test-error:0.00314
[4649]	test-error:0.003139
[4650]	test-error:0.003139
[4651]	test-error:0.003139
[4652]	test-error:0.00314
[4653]	test-error:0.003

[4923]	test-error:0.003138
[4924]	test-error:0.003138
[4925]	test-error:0.003138
[4926]	test-error:0.003137
[4927]	test-error:0.003137
[4928]	test-error:0.003138
[4929]	test-error:0.003138
[4930]	test-error:0.003137
[4931]	test-error:0.003138
[4932]	test-error:0.003138
[4933]	test-error:0.003139
[4934]	test-error:0.003138
[4935]	test-error:0.003138
[4936]	test-error:0.003138
[4937]	test-error:0.003138
[4938]	test-error:0.003138
[4939]	test-error:0.003137
[4940]	test-error:0.003137
[4941]	test-error:0.003137
[4942]	test-error:0.003138
[4943]	test-error:0.003137
[4944]	test-error:0.003138
[4945]	test-error:0.003137
[4946]	test-error:0.003137
[4947]	test-error:0.003137
[4948]	test-error:0.003137
[4949]	test-error:0.003138
[4950]	test-error:0.003138
[4951]	test-error:0.003137
[4952]	test-error:0.003137
[4953]	test-error:0.003137
[4954]	test-error:0.003137
[4955]	test-error:0.003137
[4956]	test-error:0.003137
[4957]	test-error:0.003137
[4958]	test-error:0.003137
[4959]	test-error:0.003137
[

In [ ]:
gpu_res

In [15]:
## Save Model
import joblib
#save model
joblib.dump(tst_model, 'ICC_100F') 

#load saved model
#model = joblib.load('ICL_100F')




['ICC_100F']

In [ ]:
%%time
model.eval(dtest)

In [ ]:
probas = tst_model.predict(dtest)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(probas);
################### BELOW THIS LINE IS TEST CODE ##################################

In [ ]:
tst_model.get_score(importance_type='gain')

In [ ]:
tst_model.dump_model('Vector_100_ICL.dump',fmap='xgb.fmap',with_stats=True)